In [2]:
import os
import json
from ner_eval import tokf1_calc
from collections import Counter
from seqeval.metrics import classification_report as sqclassification_report
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report as skclassification_report
import evaluate

from datasets import DatasetDict
from transformers import AutoModelForTokenClassification, AutoTokenizer, pipeline
from sghead_ner_ft import SgheadDataset, sghead_collate, sghead_evaluate_model
from mhead_ner_ft import MheadDataset, MheadTokenClassifier, mhead_collate, mhead_evaluate_model
from create_datasets import get_label_set

from nervaluate import Evaluator
import torch

cwd = os.getcwd()
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [5]:
mode = "a"
htype = "mhead"
model_name = "FacebookAI/xlm-roberta-base" #["microsoft/deberta-v3-base","FacebookAI/xlm-roberta-base","dslim/bert-base-NER-uncased"]:
r=2
dsdcts_dir = f"{cwd}/inputs/{mode}/{htype}_dsdcts"
models_dir = f"{cwd}/models/{mode}/{htype}"
results_dir = f"{cwd}/results/{mode}/{htype}/"
model_addr = f"{models_dir}/{model_name.split('/')[-1]}_{r}"
randp_fn = f"{results_dir}/randp_{model_name.split('/')[-1]}_{r}.json"
dev = "cuda"
label_list = get_label_set(mode, "mhead")

In [4]:
dataset_dict = DatasetDict.load_from_disk(f"{dsdcts_dir}/dsdct_r{r}")
tokenizer = AutoTokenizer.from_pretrained(model_name)
if htype=="sghead":
    model = AutoModelForTokenClassification.from_pretrained(model_addr).to(dev)
if htype=="mhead":
    dropout=0.1
    head_lst = get_label_set(mode, htype)
    model = MheadTokenClassifier(model_name, head_lst, dropout = dropout).to(dev)
    model.load_state_dict(torch.load(model_addr+"/model.pt", weights_only=True))
    model.eval()

In [4]:
with open(randp_fn, "r", encoding="utf-8") as f:
    randp = json.load(f)

In [1]:
seqeval = evaluate.load("seqeval")
all_reals, all_preds = [], []
sk_reals, sk_preds = [], []
sq_reals, sq_preds = [], []
for head in list(randp):
    for art in randp[head]:
        reals = [ent[0] for ent in art]
        preds = [ent[1] for ent in art]
        all_reals += reals
        all_preds += preds
        #
        reals = [ent[0] for ent in art]
        preds = [ent[1] for ent in art]
        sq_reals.append(reals)
        sq_preds.append(preds)
    res = seqeval.compute(predictions=sq_preds, references=sq_reals)
    print(head, res)
    evaluator = Evaluator(sq_reals, sq_preds, tags='BIO', loader="list")
    print(evaluator)

NameError: name 'evaluate' is not defined

In [7]:
tokf1 = tokf1_calc(htype, randp)
print(tokf1)

{'Actor': {'micro_f1': 1.0, 'macro_f1': 1.0, 'weighted_f1': 1.0}, 'InstrumentType': {'micro_f1': 1.0, 'macro_f1': 1.0, 'weighted_f1': 1.0}, 'Objective': {'micro_f1': 1.0, 'macro_f1': 1.0, 'weighted_f1': 1.0}, 'Resource': {'micro_f1': 0.9194630872483222, 'macro_f1': 0.31934731934731936, 'weighted_f1': 0.8808842164546862}, 'Time': {'micro_f1': 1.0, 'macro_f1': 1.0, 'weighted_f1': 1.0}}


In [ ]:
all_reals, all_preds = [], []
sk_reals, sk_preds = [], []
sq_reals, sq_preds = [], []
for art in randp:
    reals = [ent[0] for ent in art]
    preds = [ent[1] for ent in art]
    all_reals += reals
    all_preds += preds
    #
    reals = [ent[0][2:] if ent[0]!="O" else "O" for ent in art]
    preds = [ent[1][2:] if ent[1]!="O" else "O" for ent in art]
    sk_reals += reals
    sk_preds += preds
    #
    reals = [ent[0] for ent in art]
    preds = [ent[1] for ent in art]
    sq_reals.append(reals)
    sq_preds.append(preds)

              precision    recall  f1-score   support

           B       0.92      0.94      0.93       116
           I       0.91      0.91      0.91       138
           O       1.00      1.00      1.00      3288

    accuracy                           0.99      3542
   macro avg       0.94      0.95      0.95      3542
weighted avg       0.99      0.99      0.99      3542

              precision    recall  f1-score   support

                   0.94      0.94      0.94       254
           O       1.00      1.00      1.00      3288

    accuracy                           0.99      3542
   macro avg       0.97      0.97      0.97      3542
weighted avg       0.99      0.99      0.99      3542

              precision    recall  f1-score   support

           _       0.90      0.91      0.90       117

   micro avg       0.90      0.91      0.90       117
   macro avg       0.90      0.91      0.90       117
weighted avg       0.90      0.91      0.90       117

              preci

In [47]:
sk1 = skclassification_report(all_reals, all_preds)
sk2 = skclassification_report(sk_reals, sk_preds)
sq = sqclassification_report(sq_reals, sq_preds)
print(sk1)
print(sk2)
print(sq)

/home/marwas/miniconda3/envs/kgenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/marwas/miniconda3/envs/kgenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/marwas/miniconda3/envs/kgenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.c

                  precision    recall  f1-score   support

         B-Actor       0.85      0.90      0.88      1334
B-InstrumentType       0.55      0.62      0.58       663
     B-Objective       0.24      0.31      0.27       138
      B-Resource       0.00      0.00      0.00       119
          B-Time       0.45      0.59      0.51       178
         I-Actor       0.87      0.80      0.83      1654
I-InstrumentType       0.59      0.50      0.54       775
     I-Objective       0.59      0.29      0.39       919
      I-Resource       0.00      0.00      0.00       166
          I-Time       0.71      0.65      0.68       546
               O       0.97      0.98      0.98     70154

        accuracy                           0.95     76646
       macro avg       0.53      0.51      0.51     76646
    weighted avg       0.95      0.95      0.95     76646

                precision    recall  f1-score   support

         Actor       0.90      0.88      0.89      2988
InstrumentType

In [26]:
seqeval = evaluate.load("seqeval")
metrics = seqeval.compute(predictions=sq_preds, references=sq_reals)
print(metrics)

/home/marwas/miniconda3/envs/kgenv/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'Actor': {'precision': np.float64(0.9001026342798495), 'recall': np.float64(0.8805220883534136), 'f1': np.float64(0.8902047030959228), 'number': np.int64(2988)}, 'InstrumentType': {'precision': np.float64(0.6365561044460127), 'recall': np.float64(0.627260083449235), 'f1': np.float64(0.6318739054290718), 'number': np.int64(1438)}, 'Objective': {'precision': np.float64(0.5799373040752351), 'recall': np.float64(0.3500473036896878), 'f1': np.float64(0.43657817109144537), 'number': np.int64(1057)}, 'Resource': {'precision': np.float64(0.0), 'recall': np.float64(0.0), 'f1': np.float64(0.0), 'number': np.int64(285)}, 'Time': {'precision': np.float64(0.7012278308321964), 'recall': np.float64(0.7099447513812155), 'f1': np.float64(0.7055593685655457), 'number': np.int64(724)}, 'overall_precision': np.float64(0.7734197163368937), 'overall_recall': np.float64(0.6803758471965496), 'overall_f1': np.float64(0.7239203474555437), 'overall_accuracy': 0.9542572345588811}


In [29]:
mtcs = {
    "micro_f1":f1_score(all_reals, all_preds, average='micro'),
    "macro_f1":f1_score(all_reals, all_preds, average="macro"),
    "weighted_f1":f1_score(all_reals, all_preds, average="weighted")
}
print(mtcs)

{'micro_f1': 0.9542572345588811, 'macro_f1': 0.5149809982711112, 'weighted_f1': 0.950775895964328}


In [48]:
new_preds, new_reals = [], []
for nart in range(len(sq_reals)):
    new_preds.append(" ".join(sq_preds[nart]))
    new_reals.append(" ".join(sq_reals[nart]))

In [9]:
def metric_calc(htype, randp):
    if htype == "sghead":
        all_reals = []
        all_preds = []
        for art in randp:
            reals = [ent[0] for ent in art]
            preds = [ent[1] for ent in art]
            all_reals += reals
            all_preds += preds
        scores = {
            "sklearn":skclassification_report(all_reals, all_preds),
            "seqeval":sqclassification_report(all_reals, all_preds)
        }
        return scores
    elif htype == "mhead":
        label_lst = list(randp)
        scores_dct = {head:{} for head in label_lst}
        for head in label_lst:
            all_reals = []
            all_preds = []
            for art in randp[head]:
                reals = [ent[0] for ent in art]
                preds = [ent[1] for ent in art]
                all_reals += reals
                all_preds += preds
            scores_dct[head] = {
                "sklearn":skclassification_report(all_reals, all_preds),
                "seqeval":sqclassification_report(all_reals, all_preds)
            }
        return scores_dct

In [10]:
metrics = metric_calc(htype, randp)
print(metrics)

/home/marwas/miniconda3/envs/kgenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/marwas/miniconda3/envs/kgenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/marwas/miniconda3/envs/kgenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.c

TypeError: Found input variables without list of list.